In [14]:
# Read in Library
import pandas as pd
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import pandas as pd
from ta import add_all_ta_features
from ta import momentum


In [3]:
# Read in Price Data
csv_file = 'SPY.csv'
stock_df = pd.read_csv(csv_file)


In [4]:
# Stock Financial Data


Load Indicators

In [5]:
# Indicators
indicators_df = pd.DataFrame(index=stock_df.index)

# Add all technical indicators using TA library
indicators_df = add_all_ta_features(
    stock_df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=False
)


c:\Users\zebfr\anaconda3\envs\tradebot\Lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\zebfr\anaconda3\envs\tradebot\Lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


STRATEGIES

In [6]:
# Awesome Oscilator 0 cross
def Awesome_Oscillator_signals(stock_df):
    # Define buy and sell signals
    signals = pd.DataFrame(index=stock_df.index)
    signals['Ao_signal'] = 0  # Initialize all signals to 0

    for i in range(1, len(stock_df)):
        if (
            stock_df['momentum_ao'].iloc[i-1] < 0 and
            stock_df['momentum_ao'].iloc[i] >= 0
        ):
            signals['Ao_signal'].iloc[i] = 1  # Buy signal
        elif (
            stock_df['momentum_ao'].iloc[i-1] >= 0 and
            stock_df['momentum_ao'].iloc[i] < 0
        ):
            signals['Ao_signal'].iloc[i] = -1  # Sell signal
    return signals

ao_signals = pd.DataFrame(Awesome_Oscillator_signals(stock_df))
print(ao_signals)


      Ao_signal
0             0
1             0
2             0
3             0
4             0
...         ...
7636          0
7637          0
7638          0
7639          0
7640          0

[7641 rows x 1 columns]


In [16]:
# KAMA Fast/Slow Cross
def kama_cross_signals(stock_df, fast_period=10, slow_period=20):
    signals = pd.DataFrame(index=stock_df.index)
    signals['kama_signal'] = 0

    # Calculate Fast KAMA
    fast_kama = momentum.kama(stock_df['Close'], window=fast_period)

    # Calculate Slow KAMA
    slow_kama = momentum.kama(stock_df['Close'], window=slow_period)
    for i in range(1, len(stock_df)):
        if fast_kama[i] > slow_kama[i] and fast_kama[i - 1] <= slow_kama[i - 1] and stock_df['Close'][i] > fast_kama[i]:
            signals['kama_signal'].iloc[i] = 1  # Buy signal (fast KAMA above slow KAMA and price above fast KAMA)
        elif fast_kama[i] < slow_kama[i] and fast_kama[i - 1] >= slow_kama[i - 1] and stock_df['Close'][i] < fast_kama[i]:
            signals['kama_signal'].iloc[i] = -1  # Sell signal (fast KAMA below slow KAMA and price below fast KAMA)
        elif stock_df['Close'][i] > fast_kama[i] and stock_df['Close'][i - 1] <= fast_kama[i - 1]:
            stock_df.loc[i, 'Signal'] = 1  # Buy signal (price crosses above KAMA)
        elif stock_df['Close'][i] < fast_kama[i] and stock_df['Close'][i - 1] >= fast_kama[i - 1]:
            stock_df.loc[i, 'Signal'] = -1

    return signals

kama_signals = pd.DataFrame(kama_cross_signals(stock_df, fast_period=10, slow_period=20))
print(kama_signals)

      kama_signal
0               0
1               0
2               0
3               0
4               0
...           ...
7636            0
7637            0
7638            0
7639            0
7640            0

[7641 rows x 1 columns]


In [ ]:
# PPO

def ppo_signals(stock_data, fast_window=12, slow_window=26, signal_window=9):
    signals = pd.DataFrame(index=stock_data.index)
    signals['PPO_signal'] = 0

    ppo = momentum.PercentagePriceOscillator(stock_data['Close'], fast_window, slow_window, signal_window)

    ppo_values = ppo.ppo()
    ppo_signal = ppo.ppo_signal()

    # Generate buy (1) and sell (-1) signals based on PPO and its signal line
    for i in range(1, len(stock_data)):
        if ppo_values[i] > ppo_signal[i] and ppo_values[i - 1] <= ppo_signal[i - 1]:
            signals.loc[stock_data.index[i], 'PPO_signal'] = 1  # Buy signal (PPO crosses above signal line)
        elif ppo_values[i] < ppo_signal[i] and ppo_values[i - 1] >= ppo_signal[i - 1]:
            signals.loc[stock_data.index[i], 'PPO_signal'] = -1  # Sell signal (PPO crosses below signal line)

    return signals

ppo_signals = pd.DataFrame(ppo_signals(stock_df))
print(ppo_signals)

In [7]:
# Analyst Rating and Events


In [8]:
# Pre-process Data


In [9]:
# Models


In [10]:
# Predict Buy/Sell Condition


In [11]:
# Evaluate
